<a href="https://colab.research.google.com/github/seadreamjj/danmu-ID/blob/main/%E7%8C%AB%E8%80%B3%E5%BC%B9%E5%B9%95id%E8%AE%A1%E7%AE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import requests
import re
import pandas as pd

#输入剧集地址
drama_id ='   https://www.missevan.com/mdrama/76879   '

drama_url = 'http://www.missevan.com/dramaapi/getdrama?drama_id=  '
url = drama_url +drama_id.split("/")[4]
drama_result =requests.get(url).text
pattern1 =re.compile('sound_id":(.+?),"type')
pattern2 = re.compile('"need_pay":(.+?),"')
sound_id = re.findall(pattern1, drama_result)
pay_type = re.findall(pattern2, drama_result)
pay_type =pay_type[1:]
drama = pd.DataFrame({'sound_id': sound_id,'pay_type':pay_type})
sound_id =list(drama['sound_id'][drama['pay_type']=='1'])
sound_url = 'http://www.missevan.com/sound/getdm?soundid='

for i in range(0,len(sound_id)):
  url=sound_url+sound_id[i]
  sound_result=requests.get(url).text
  pattern3=re.compile(r'p="(.+?)"')
  dm =re.findall(pattern3, sound_result)
  if i==0:
    dm1=pd.DataFrame({'dm':dm})
    dms=dm1['dm'].str.split('>',expand=True)[0]
    uid =dms.str.split(',',expand=True)[6]
    dm1['uid']=uid
  else:
    dm2=pd.DataFrame({'dm':dm})
    dms_i=dm2['dm'].str.split('>',expand=True)[0]
    uid =dms.str.split(',',expand=True)[6]
    dm2['uid']=uid
    dm1=dm1.append(dm2)

print("本剧实时付费ID为：", len(dm1['uid'].unique()))
print("本剧实时付费弹幕数量为：",len(dm1['uid']))

AttributeError: 'DataFrame' object has no attribute 'append'

猫耳付费id抓取，正确版


In [8]:
import requests
import re
import pandas as pd

# 输入剧集地址
drama_id = 'https://www.missevan.com/mdrama/74274'

# 构建 URL
drama_url = 'http://www.missevan.com/dramaapi/getdrama?drama_id='
url = drama_url + drama_id.split("/")[4]

# 请求剧集数据
drama_result = requests.get(url).text

# 使用正则表达式提取 sound_id 和 pay_type
pattern1 = re.compile(r'"sound_id":(\d+),')  # 提取 sound_id
pattern2 = re.compile(r'"need_pay":(\d+),')  # 提取付费状态 need_pay
sound_id = re.findall(pattern1, drama_result)
pay_type = re.findall(pattern2, drama_result)

# 去掉第一个非付费项
pay_type = pay_type[1:]

# 创建 DataFrame
drama = pd.DataFrame({'sound_id': sound_id, 'pay_type': pay_type})

# 筛选付费 sound_id
sound_id = list(drama['sound_id'][drama['pay_type'] == '1'])

sound_url = 'http://www.missevan.com/sound/getdm?soundid='
dm1 = pd.DataFrame()

# 循环处理每个 sound_id，获取弹幕信息
for i in range(len(sound_id)):
    url = sound_url + sound_id[i]
    sound_result = requests.get(url).text

    # 提取弹幕内容
    pattern3 = re.compile(r'p="(.+?)"')
    dm = re.findall(pattern3, sound_result)

    # 将弹幕数据存储到 DataFrame
    dm_temp = pd.DataFrame({'dm': dm})
    dms_split = dm_temp['dm'].str.split('>', expand=True)[0]
    uid = dms_split.str.split(',', expand=True)[6]
    dm_temp['uid'] = uid

    # 合并数据，使用 pd.concat() 替代 append()
    if i == 0:
        dm1 = dm_temp
    else:
        dm1 = pd.concat([dm1, dm_temp], ignore_index=True)

# 输出不重复的 UID 数量和总弹幕数量
print("本剧实时付费ID为：", len(dm1['uid'].unique()))
print("本剧实时付费弹幕数量为：", len(dm1['uid']))


本剧实时付费ID为： 550
本剧实时付费弹幕数量为： 17520


# 新段落

漫播弹幕id抓取


In [12]:
import requests
import re
import json

# 输入漫播剧集的任意一集链接（PC 或 APP 端）
url = "https://www.kilamanbo.com/manbo/pc/detail?id=1937619915414438001&collectId=1945864036419633309"

# 判断链接类型
if 'pc' in url:
    detail_id = url.split('Id=')[1].split('&')[0]  # 提取 detail_id
    new_url = "https://manbo.hongdoulive.com/web_manbo/dramaSetDetail?dramaSetId=" + detail_id
    temp = 'data'
else:
    episode_id = url.split('id=')[1][:19]  # 提取 episode_id
    radio_id = url.split('DramaId=')[1][:19]  # 提取 radio_id
    new_url = "https://manbo.hongrenshuo.com.cn/api/v207/radio/drama/set/h5/detail?radioDramaSetId=%s&radioDramaId=%s" % (episode_id, radio_id)
    temp = 'b'


r= requests.get(new_url).text
r= str(json.loads(r)[temp]['radioDramaResp']['setRespList'])
# 请求新的 URL，获取剧集信息
#response = requests.get(new_url).text

# 解析 JSON 数据，提取付费集信息
#r = json.loads(response)
pattern1 = re.compile("'vipFree': (.+?),")
vipFree = re.findall(pattern1, r)
num = len(vipFree)

pattern2 = re.compile("'setId': (.+?),")
setId = re.findall(pattern2, r)

# 找到所有免费集或付费集的 ID
episodes = [setId[i] for i in range(num) if vipFree[i] == '1']

# 如果没有免费集，查找付费集
if len(episodes) == 0:
    pattern3 = re.compile("'payType': (.+?),")
    payType = re.findall(pattern3, r)
    episodes = [setId[i] for i in range(num) if payType[i] == '1']

# 弹幕列表
danmu = []

# 获取每集的弹幕信息
for each in episodes:
    next_url = "https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=%s&startTime=0&endTime=10000000" % each
    print(next_url)
    danmakuList = json.loads(requests.get(next_url).content)['b']['danmakuList']

    # 提取每条弹幕的 eid
    for j in danmakuList:
        danmu.append(j['eid'])

# 输出不重复的弹幕 ID 数量和总弹幕数量
print("付费集ID数：%d" % len(set(danmu)))
print("付费集弹幕数：%d" % len(danmu))


https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1952275658088382525&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1954804814005141586&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1957461100953862215&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1960056287697830140&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1962697177188794464&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1962696064792264745&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/api/v11/radio/drama/set/danmaku/h5/pull?radioDramaSetId=1963435314563252306&startTime=0&endTime=10000000
https://manbo.hongrenshuo.com.cn/a

调整漫播数据，只保留正剧（去掉付费集花絮）

试图计算猫耳收费剧集的平均时长


In [75]:
import requests
import re
import pandas as pd

# 输入剧集地址
drama_id = 'https://www.missevan.com/mdrama/77685'

# 构建 URL
drama_url = 'http://www.missevan.com/dramaapi/getdrama?drama_id='
url = drama_url + drama_id.split("/")[4]

# 请求剧集数据
drama_result = requests.get(url).text

# 使用正则表达式提取 sound_id、pay_type 和 duration
pattern1 = re.compile(r'"sound_id":(\d+),')  # 提取 sound_id
pattern2 = re.compile(r'"need_pay":(\d+),')  # 提取付费状态 need_pay
pattern3 = re.compile(r'"duration":(\d+),')  # 提取时长 duration (秒)

sound_id = re.findall(pattern1, drama_result)
pay_type = re.findall(pattern2, drama_result)
durations = re.findall(pattern3, drama_result)

# 打印各列表的长度，检查是否一致
print("sound_id 长度:", len(sound_id))
print("pay_type 长度:", len(pay_type))
print("durations 长度:", len(durations))

# 去掉第一个非付费项（通常第一个为非付费）
pay_type = pay_type[1:]
durations = durations[1:]

# 确保长度一致后才能继续创建 DataFrame
min_length = min(len(sound_id), len(pay_type), len(durations))

# 截断为相同的长度
sound_id = sound_id[:min_length]
pay_type = pay_type[:min_length]
durations = durations[:min_length]

# 创建 DataFrame，包含 sound_id、pay_type 和 duration
drama = pd.DataFrame({
    'sound_id': sound_id,
    'pay_type': pay_type,
    'duration': [int(dur) for dur in durations]  # 将时长转换为整数
})

# 筛选付费集
paid_drama = drama[drama['pay_type'] == '1']

# 计算平均时长（秒 -> 分钟）
average_duration_minutes = paid_drama['duration'].mean() / 60

# 输出收费集的平均时长（分钟）
print("收费集的平均时长为：%.2f 分钟" % average_duration_minutes)


sound_id 长度: 26
pay_type 长度: 27
durations 长度: 26
收费集的平均时长为：21307.96 分钟


算平均时长

In [27]:
import requests
import re
import pandas as pd

# 输入猫耳剧集的 URL
drama_id = 'https://www.missevan.com/mdrama/67153'

# 构建 API URL
drama_url = 'http://www.missevan.com/dramaapi/getdrama?drama_id='
url = drama_url + drama_id.split("/")[4]

# 请求剧集数据
drama_result = requests.get(url).text

# 使用正则表达式提取 sound_id 和 pay_type
pattern1 = re.compile(r'"sound_id":(\d+),')  # 提取 sound_id
pattern2 = re.compile(r'"need_pay":(\d+),')  # 提取付费状态 need_pay
sound_id = re.findall(pattern1, drama_result)
pay_type = re.findall(pattern2, drama_result)

# 去掉第一个非付费项
pay_type = pay_type[1:]

# 创建 DataFrame，筛选付费集
drama = pd.DataFrame({'sound_id': sound_id, 'pay_type': pay_type})
paid_sound_id = list(drama['sound_id'][drama['pay_type'] == '1'])

# 定义获取音频时长的函数
def get_audio_duration(sound_id):
    # 构建 avinfo 请求 URL
    audio_url = f"https://www.missevan.com/sound/getsound?soundid={sound_id}&avinfo"
    response = requests.get(audio_url).json()

    # 检查 'format' 字段是否存在
    if 'format' in response:
        duration = float(response['format'].get('duration', 0))
        return duration / 60  # 返回分钟数
    # 如果 'format' 字段不存在，检查 'streams' 字段
    elif 'streams' in response:
        stream_info = response['streams'][0]  # 通常时长信息在第一个流中
        if 'duration' in stream_info:
            duration = float(stream_info.get('duration', 0))
            return duration / 60  # 返回分钟数
        else:
            raise KeyError("'streams' 字段中缺少时长信息")
    else:
        raise KeyError("'format' 和 'streams' 字段都缺失")

# 获取每个收费音频的时长
durations = []
for sid in paid_sound_id:
    try:
        duration = get_audio_duration(sid)
        durations.append(duration)
        print(f"音频 sound_id: {sid}, 时长: {duration:.2f} 分钟")
    except Exception as e:
        print(f"获取 sound_id {sid} 时长时出错: {e}")

# 计算收费集的平均时长
if durations:
    avg_duration = sum(durations) / len(durations)
    print(f"收费集的平均时长: {avg_duration:.2f} 分钟")
else:
    print("没有获取到收费集的时长信息。")


获取 sound_id 8436631 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8473659 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8511196 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8510859 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8540011 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8572502 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8605095 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8604873 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8637984 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8667760 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8698993 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8703288 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8730391 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8762298 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8792924 时长时出错: "'format' 和 'streams' 字段都缺失"
获取 sound_id 8796330 时长时出错: "'format' 和 'streams' 字段都缺失"
没有获取到收费集的时长信息。
